In [4]:
!pip install --quiet -U lancedb
!pip install --quiet gradio


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [6]:
import duckdb
import lancedb

uri = "~/datasets/demo"
db = lancedb.connect(uri)
tbl = db.open_table("diffusion")

# Initialize CLIP model
def embed_func(vector_query):
    pass

def find_image_vectors(query):
    emb = embed_func(vector_query)
    return _extract(tbl.search(emb).limit(9).to_df())

def find_image_keywords(query):
    return _extact(tbl.search(keyword_query).limit(9).to_df())

def find_image_sql(query):
    imagenet_1k = tbl.to_lance()
    return _extract(duckdb.query(sql_query).to_df())

def _extract(df):
    image_col = "image column name"
    label_col = "label column name"
    return [(row[image_col], row[label_col]) for row in df.iterrows()]    

In [11]:
import gradio as gr


with gr.Blocks() as demo:
    gallery = gr.Gallery(
            label="Found images", show_label=False, elem_id="gallery"
        ).style(columns=[3], rows=[3], object_fit="contain", height="auto")
    with gr.Row():
        with gr.Tab("Embeddings"):
            vector_query = gr.Textbox(value="people riding bikes")
            gr.Button("Submit").click(find_image_vectors, inputs=vector_query, outputs=gallery)
        with gr.Tab("Keywords"):
            keyword_query = gr.Textbox(value="turtle")
            gr.Button("Submit").click(find_image_keywords, inputs=keyword_query, outputs=gallery)
        with gr.Tab("SQL"):
            sql_query = gr.Textbox(value="SELECT * from imagenet_1k WHERE label='crayfish' LIMIT 9")
            gr.Button("Submit").click(find_image_sql, inputs=sql_query, outputs=gallery)
    with gr.Row():
        gallery
        
demo.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
